In [1]:
import pandas as pd
from pandas import  DataFrame
from sklearn.model_selection import train_test_split

In [33]:
df = pd.read_csv('stal_trimmed.csv', encoding = "ISO-8859-1")

df['Type'].value_counts()

Others      796
Lockouts    270
Name: Type, dtype: int64

In [34]:
X = df['Description']
y = df['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [35]:
# import and initiate TfidfVectorizer (with the default parameters)
# use TreeankWordTokenizer
from nltk.tokenize import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vect = CountVectorizer()
vect.set_params(tokenizer=tokenizer.tokenize)

# remove English stop words
vect.set_params(stop_words='english')

# include 1-grams and 2-grams
vect.set_params(ngram_range=(1,2))

# ignore terms that appear in more than 50% of the documents
vect.set_params(max_df=0.5)

# only keep terms that appear in at least 2 documents
vect.set_params(min_df=2)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000000000DA232B0>>,
        vocabulary=None)

In [36]:
# learn the 'vocabulary' of the training data
vect.fit(X_train)

# examine the fitted vocabulary
#vect.get_feature_names()[:50]

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=2,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000000000DA232B0>>,
        vocabulary=None)

In [37]:
train_vectors = vect.transform(X_train)
test_vectors = vect.transform(X_test)

In [38]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = GaussianNB()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [39]:
rf.fit(train_vectors.toarray(), y_train)

#rf.fit(train_vectors.toarray(), y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [40]:
from sklearn.metrics.classification import classification_report, accuracy_score
from sklearn.cross_validation import cross_val_predict

# generate your cross-validation prediction with 10 fold Stratified sampling
y_pred = cross_val_predict(rf, test_vectors.toarray(), y_test, cv=10)

print(classification_report(y_test, y_pred))
print("ACCURACY::", accuracy_score(y_pred, y_test))


             precision    recall  f1-score   support

   Lockouts       0.97      0.83      0.90        47
     Others       0.95      0.99      0.97       167

avg / total       0.96      0.96      0.96       214

('ACCURACY::', 0.95794392523364491)


In [54]:
#val = pd.read_csv('val.csv', encoding="ISO-8859-1")
val = ['Customers account was locked']
val_vector = vect.transform(val)

rf.predict(val_vector.toarray())

array([u'Lockouts'], dtype=object)